In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.listdir("../input/")

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, Flatten, ZeroPadding2D, Rescaling
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

In [ ]:
train_df = pd.read_csv("/kaggle/input/humpback-whale-identification/train.csv")

In [ ]:
train_df['Id'] = train_df['Id'].astype('category').cat.codes

In [ ]:
test_df = pd.read_csv("/kaggle/input/humpback-whale-identification/sample_submission.csv")

In [ ]:
train_df.head()

In [ ]:
INP_SIZE      = (512, 512) 
TARGET_SIZE   = (224, 224) 
INTERPOLATION = "bilinear"
N_CLASSES = 15587

NUM_FOLDS  = 5
BATCH_SIZE = 24
SEED       = 42

DATA_DIR  = '../input/humpback-whale-identification/'
TRAIN_DIR = DATA_DIR + 'train/'
TEST_DIR  = DATA_DIR + 'test/'

In [ ]:
# SetAutoTune
AUTOTUNE = tf.data.AUTOTUNE 
#https://www.kaggle.com/ipythonx/tf-keras-learning-to-resize-image-for-vit-model/notebook
def build_augmenter(is_labelled):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_saturation(img, 0.65, 1.05)
        img = tf.image.random_brightness(img, 0.05)
        img = tf.image.random_contrast(img, 0.75, 1.05)
        img = tf.image.random_hue(img, 0.05)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    return augment_with_labels if is_labelled else augment

def build_decoder(is_labelled, size):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        img = tf.image.decode_jpeg(file_bytes, channels = 3)
        img = tf.image.resize(img, (size[0], size[1]))
        return tf.cast(tf.divide(img, 255.),tf.float32)
    
    def decode_with_labels(path, label):
        label = tf.cast(label, tf.int32)
        return decode(path),label
    
    return decode_with_labels if is_labelled else decode

def create_dataset(df, 
                   batch_size  = 32, 
                   is_labelled = False, 
                   augment     = False, 
                   repeat      = False, 
                   shuffle     = False,
                   size        = TARGET_SIZE):
    decode_fn    = build_decoder(is_labelled, size)
    augmenter_fn = build_augmenter(is_labelled)
    
    # Create Dataset
    if is_labelled:
        dataset = tf.data.Dataset.from_tensor_slices((df['Image'].values, df['Id'].values))
    else:
        dataset = tf.data.Dataset.from_tensor_slices((df['Image'].values))
        
    dataset = dataset.map(decode_fn, num_parallel_calls = AUTOTUNE)
    dataset = dataset.map(augmenter_fn, num_parallel_calls = AUTOTUNE) if augment else dataset
    dataset = dataset.repeat() if repeat else dataset
    dataset = dataset.shuffle(1024, reshuffle_each_iteration = True) if shuffle else dataset
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
train_df['Image'] = train_df['Image'].apply(lambda x: f'{TRAIN_DIR}{x}')

In [ ]:
train_ds = create_dataset(train_df, 32, True)

In [ ]:
train_ds

In [ ]:
test_df['Image'] = test_df['Image'].apply(lambda x: f'{TEST_DIR}{x}')
test_ds = create_dataset(test_df)

In [ ]:
test_ds